In [ ]:
import torch
import pandas as pd
import astropy as ap
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
# from astropy.io import fits
import pdb
from scipy.ndimage.filters import maximum_filter1d
import glob
import fitsio as fits
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data_utils
from torch.distributions import Bernoulli, Binomial, Normal, Uniform
from torch.nn.utils import clip_grad_norm_
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence
from torch.distributions.normal import Normal
from tqdm import tqdm
import math

In [ ]:
L = 1

In [ ]:
T = 0.5

In [ ]:
math.pi

In [ ]:
x = torch.arange(0, L, 0.02)

In [ ]:
y = torch.sin(2*math.pi*x/T)

In [ ]:
sns.lineplot(x=x.numpy(), y=y.numpy())

In [ ]:
Ts = [L/2, L/4, L/8]
Ts = [L/8]

In [ ]:
def make_sin():
    t_obs = x[:len(x)//2]
    t_pred = x[len(x)//2:]
#     T = Normal(loc=L/2, scale=L/4).sample()
    T = random.choice(Ts)
    y = 1*torch.sin(2*math.pi*x/T)
    y1 = y[:len(x)//2]
    y2 = y[len(x)//2:]
    p = Uniform(0.5, 0.9)
    measurements = Binomial(probs=p.sample()).sample(t_obs.size())
    y_measure = y1 * measurements
    y_out = torch.stack((t_obs, t_pred, y_measure, y2))
    return y_out

In [ ]:
yy = make_sin()

In [ ]:
yy.shape

In [ ]:
# sns.lineplot(x=x.numpy(), y=make_sin().numpy())

In [ ]:
# sns.scatterplot(x=x.numpy(), y=make_sin().numpy())

### Build dataset

In [ ]:
def make_set(n=1000):
    samples = []
    for _ in range(n):
        samples.append(make_sin())
    return samples

In [ ]:
samples = make_set()

In [ ]:
test=samples[10]

In [ ]:
sns.scatterplot(x=test[1].numpy(), y=test[3].numpy())

### Output

In [ ]:
outp = torch.stack(samples)

In [ ]:
outp.shape

In [ ]:
# train = outp[:180]
# cv = outp[180:]
train = outp[:900]
cv = outp[900:]

In [ ]:
train.shape, cv.shape

In [ ]:
from torch.utils.data import DataLoader
from utils import collate_ts, batchify, collate_extrap
from loading import ToyDataset

In [ ]:
train_set = ToyDataset(train)
cv_set = ToyDataset(cv)

In [ ]:
train_loader = DataLoader(train_set, batch_size=150, collate_fn=collate_extrap)
cv_loader = DataLoader(cv_set, batch_size=150, collate_fn=collate_extrap)

In [ ]:
torch.save(train_loader, 'toy_train_extrap.pt')
torch.save(cv_loader, 'toy_cv_extrap.pt')

### Checks

In [ ]:
l = torch.load('toy_train_extrap.pt')

In [ ]:
d = next(iter(l))

In [ ]:
d['tp_to_predict']